In [108]:
import numpy as np
import rasterio
from rasterio.plot import show
import os
import matplotlib.pyplot as plt
import rioxarray
import tifffile
from PIL import Image

In [109]:
root = '/home/tiago/research/greenai/split_dataset/valdoeiro/x7'

dsm_dir = os.path.join(root,'dsm')
image_dir = os.path.join(root,'images')
mask_dir = os.path.join(root,'masks')

if not os.path.isfile(dsm_dir) or not os.path.isfile(image_dir) or not os.path.isfile(mask_dir):
    NameError

In [ ]:
from utils import utils

image_files = utils.get_files(image_dir)
image_files['files'] = sorted(image_files['files'])
dsm_files = utils.get_files(dsm_dir)
dsm_files['files'] = sorted(dsm_files['files'])
mask_files = utils.get_files(mask_dir)
mask_files['files'] = sorted(mask_files['files'])

In [114]:
import tifffile
def show_triplet(img,dsm,mask):
    '''
    Input PILs 
    '''
    
    img = Image.fromarray(img)
    dsm = Image.fromarray(dsm)
    mask= Image.fromarray(mask)
    
    dst = Image.new('RGB', (img.width + dsm.width + mask.width, mask.height))
    dst.paste(img, (0, 0))
    dst.paste(dsm, (img.width, 0))
    dst.paste(mask, (img.width + dsm.width, 0))
    dst.show()

def get_file(files,index):
    return(os.path.join(files['root'],files['files'][index] + '.' + files['file_type']))

def get_triplet(img,dsm,mask,i):
    img_file  =  get_file(img,i)
    dsm_file  =  get_file(dsm,i)
    mask_file =  get_file(mask,i)
    return(img_file,dsm_file,mask_file)

def load_file(file):
    file_type = file.split('.')[-1]
    if file_type == 'npy':
        im = np.load(file)
    else:
        im = tifffile.imread(file)
    return(im)

def load_triplet(im_file,dsm_file,mask_file):
    
    img = load_file(im_file)

    dsm = load_file(dsm_file)

    mask = load_file(mask_file)


    dsm = (dsm-dsm.min())
    dsm = dsm/dsm.max()*255
    
    if len(dsm.shape)<3:
        dsm = np.expand_dims(dsm,axis=-1)
    
    dsm = np.concatenate((dsm,dsm,dsm),axis=2)

    if len(mask.shape)<3:
        mask = np.expand_dims(mask,axis=-1)

    if mask.shape[-1]<3:
        mask = np.concatenate((mask,mask,mask),axis=2)

    return(img,dsm,mask)



DSM Files

In [111]:
from utils import utils

image_files = utils.get_files(image_dir)
image_files['files'] = sorted(image_files['files'])
dsm_files = utils.get_files(dsm_dir)
dsm_files['files'] = sorted(dsm_files['files'])
mask_files = utils.get_files(mask_dir)
mask_files['files'] = sorted(mask_files['files'])

In [115]:
from PIL import Image
import numpy as np
frame = []
n_files = len(image_files['files'])
for i in range(20):
    img_file, dsm_file, mask_file = get_triplet(image_files,dsm_files,mask_files,i)
    img,dsm,mask = load_triplet(img_file,dsm_file,mask_file)
    iamge_row = np.hstack((img,dsm,mask))
    frame.append(iamge_row)

frame = np.vstack(frame).astype(np.uint8)
frame_pil = Image.fromarray(frame)
frame_pil.show()